# Classes feitas até aqui para o trabalho

In [ ]:
!sudo apt update && sudo apt install ffmpeg -y
!pip install openai-whisper
!pip install telebot
!pip install openai-whisper


###Bibliotecas usadas:

In [3]:
import telebot
import io
import tensorflow as tf
import tensorflow_hub as hub
import librosa
import whisper
import csv

### Classificação do áudio:

In [2]:
class ClassificadorAudio:
    def __init__(self):
        print("Carregando YAMNet...")
        self.model = hub.load('https://tfhub.dev/google/yamnet/1')
        class_map_path = self.model.class_map_path().numpy().decode('utf-8')
        self.class_names = self._ler_labels(class_map_path)

    def _ler_labels(self, path):
        classes = []
        with tf.io.gfile.GFile(path) as f:
            reader = csv.DictReader(f)
            for row in reader:
                classes.append(row['display_name'])
        return classes

    def identificar_som(self, arquivo_memoria):
        arquivo_memoria.seek(0)
        audio, sr = librosa.load(arquivo_memoria, sr=16000, mono=True)
        scores, _, _ = self.model(audio)
        media_scores = tf.reduce_mean(scores, axis=0)
        idx_max = tf.argmax(media_scores)
        return self.class_names[idx_max], media_scores[idx_max].numpy()


###Transcrição de áudio:

In [4]:
class TranscritorWhisper:
    def __init__(self):
        modelo = input("Modelo do Whisper (large/base):")
        print(f"Carregando Whisper (modelo {modelo})...")
        self.model = whisper.load_model(modelo)

    def transcrever(self, arquivo_memoria):
        arquivo_memoria.seek(0)
        audio, sr = librosa.load(arquivo_memoria, sr=16000)
        resultado = self.model.transcribe(audio=audio, fp16=False)
        arquivo_memoria.close()
        return resultado['text']

### Classificação da imagem:

Precisamos de uma classe que:
* No construtor, carregue o modelo yolo a ser usado.
* Tenha um método (def classificar_imagem) que recebe uma imagem (arquivo_memoria) e retorne outra imagem já classificada e uma legenda sobre o que foi classificado (opcional).

In [ ]:
class ClassificadorImagem:
    def __init__(self):
        print("Carregando YOLOv5...")
    def classificar_imagem(self, arquivo_memoria):

        return arquivo_memoria, legenda


### Base do Bot

In [5]:
class BotTelegram:
    def __init__(self, token):
        self.__token = token  # Atributo privado
        self.bot = telebot.TeleBot(self.__token)
    def _configurar_handlers(self):

        @self.bot.message_handler(commands=['start'])
        def start(message):
            nome = message.from_user.first_name
            self.bot.reply_to(message, f"Olá, {nome}! Sou um BOT criado para transcrever seus áudios e classificar áudios e imagens.")

        @self.bot.message_handler(commands=['ajuda'])
        def ajuda(message):
            self.bot.reply_to(message, "Experimente me enviar uma imagem ou áudio e eu vou tentar classificá-lo.")

        @self.bot.message_handler(content_types=['video', 'sticker', 'animation', 'video_note', 'document', 'poll'])
        def responder(message):
            self.bot.reply_to(message, "Desculpe, não trabalhamos com esse tipo de arquivo. Tente nos enviar uma imagem ou um áudio.")

        @self.bot.message_handler(content_types=['text'])
        def texto(message):
            self.bot.reply_to(message, "Desculpe, não consigo entender o que você diz. Não sou adaptado para compreender e responder textos. Eu apenas consigo classificar e/ou transcrever imagens e áudios que você me enviar.")

    def executar(self):
        print("Bot online.")
        self.bot.infinity_polling()

###Bot para o áudio:

In [6]:
class BotAudio(BotTelegram):
    def __init__(self, token, motor_som, motor_texto):
        super().__init__(token)

        self.classificador = motor_som
        self.transcritor = motor_texto
        self._configurar_handlers()

    def _configurar_handlers(self):
        super()._configurar_handlers()
        @self.bot.message_handler(content_types=['voice', 'audio'])
        def tratar_audio(message):
            status = self.bot.reply_to(message, "⏳ Analisando áudio...")
            try:
                file_info = self.bot.get_file(message.voice.file_id if message.voice else message.audio.file_id)
                downloaded_file = self.bot.download_file(file_info.file_path)
                audio_ram = io.BytesIO(downloaded_file)

                classe, confianca = self.classificador.identificar_som(audio_ram)
                resposta = f"✅ Som identificado: {classe} ({confianca:.2%})\n"

                if "Speech" in classe or "Conversation" in classe:
                    self.bot.edit_message_text(resposta + "🎙️ Transcrevendo...", message.chat.id,
                                               status.message_id)
                    texto = self.transcritor.transcrever(audio_ram)
                    resposta += f"\n📝 Transcrição: {texto}"

                self.bot.edit_message_text(resposta, message.chat.id, status.message_id, parse_mode="Markdown")

            except Exception as e:
                resposta += f"\nErro: {str(e)}"
                self.bot.edit_message_text(resposta, message.chat.id, status.message_id, parse_mode="Markdown")

            finally:
                audio_ram.close()

###Bot para a imagem:

Depois que a classe ClassificadorImagem estiver pronta, retirar os #'s dos códigos comentados da classe BotImagem

In [ ]:
class BotImagem(BotTelegram):
def __init__(self, token, motor_imagem):
        super().__init__(token)
        self.classificador = motor_imagem
        self._configurar_handlers()

def classificar_imagem(self):
        @self.bot.message_handler(content_types=['photo'])
        def handle_photos(message):
            status = self.bot.reply_to(message, "⏳ Imagem recebida. Vamos tentar classificá-la.\n\nBaixando imagem...")

            file_id = message.photo[-1].file_id # Pega a maior resolução
            file_info = self.bot.get_file(file_id)
            downloaded_file = self.bot.download_file(file_info.file_path)

            imagem_ram = io.BytesIO(downloaded_file)

            self.bot.edit_message_text("Download concuído.\n\nClassificando imagem...", message.chat.id, status.message_id)

            #imagem_ram, legenda = self.classificador.classificar_imagem(imagem_ram)  #quando tiver a classe de imagens

            self.bot.delete_message(message.chat.id, status.message_id)
            self.bot.send_photo(message.chat.id, imagem_ram, caption = legenda)
            imagem_ram.close()


##BotAdmin

In [ ]:
class BotAdmin(BotAudio, BotImagem):
    def __init__(self, token, motor_som, motor_texto, motor_imagem):
        super().__init__(token, motor_som, motor_texto, motor_imagem)

##Execução:

In [8]:
if __name__ == "__main__":
    TOKEN = '7812768588:AAFyWirqLGt_B-cdMip7sWMp8n_bVBYvzyo'

    # 1. Instancia as IAs primeiro
    ia_som = ClassificadorAudio()
    ia_texto = TranscritorWhisper()
    #Adicionar: ia_imagem = ClassificadorImagem()

    # 2. Instancia a classe filha, que já configura a classe mãe internamente
    # 2.1 Ao trocar as coisas pelo descrito no comentário abaixo: Instancia uma classe que configura as suas classes mães (BotAudio e BotImagem) e sua classe "avó" (BotTelegram).
    meu_bot = BotAudio(TOKEN, ia_som, ia_texto)#Adicionar ia_imagem nos atributos e trocar BotAudio por BotAdmin

    # 3. Inicia o polling
    meu_bot.executar()

Carregando YAMNet...
Modelo do Whisper (large/base):base
Carregando Whisper (modelo base)...
Bot online.


2025-12-27 16:46:22,355 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2025-12-27 16:46:22,370 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling


Modelo Whisper: Adicionei a opção de escolha entre os modelos large e base para o operador que irá iniciar o bot. O modelo large é muito preciso, mas é lento e pesado. O google colab suporta ele no limite da memória RAM disponível e um bug ou recompilação reinicia a sessão. O modelo base é inferior, mas é significativamente mais leve e veloz.

Anotações:

* Classes prontas para uso (depois de tirar os códigos dos comentários):

ClassificadorAudio

TranscritorWhisper

BotTelegram

BotAudio(BotTelegram)

Main

* Classes que precisam de ser testadas:

BotAdmin

BotImagem(BotTelegram)

* Classes que faltam fazer:

ClassificadorImagem

(e outros complementos necessários para classificar as imagens)

* Também falta adicionar as bibliotecas para as imagens
